<a href="https://colab.research.google.com/github/CharleneLimKH/GenQ-Hackathon/blob/main/GenQP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install qiskit-optimization qiskit-algorithms
!pip install networkx
!pip install qiskit
%pip install fire-opal qiskit matplotlib qiskit-ibm-runtime qctrl-visualizer -q


In [20]:
import json
import networkx as nx
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
import itertools
gr= nx.Graph()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io


# Define the run function which will be the entry point
def run(input_data, solver_params=None, extra_arguments=None):
    # Step 1: Read the input data
    nodes = input_data['nodes']
    adjacency_matrix = input_data['adjacency_matrix']
    target_edges = input_data['target_edges']
    optional_edges = input_data['optional_edges']

    # Step 2: Create the graph based on input
    G = nx.Graph()
    G.add_nodes_from(nodes)

    # Step 3: Add edges from the adjacency matrix
    n = len(nodes)
    node_indices = {i: node for i, node in enumerate(nodes)}  # Map index to node label
    for i in range(n):
        for j in range(n):
            if adjacency_matrix[i][j] > 0:
                u = node_indices[i]
                v = node_indices[j]
                G.add_edge(u, v, weight=adjacency_matrix[i][j])

    # Step 4: Create an edge list and a mapping from edges to indices
    edge_list = list(G.edges())
    edge_to_index = {}
    for i, edge in enumerate(edge_list):
        u, v = edge
        edge_sorted = tuple(sorted((u, v)))
        edge_to_index[edge_sorted] = i

    # Step 5: Create QUBO problem
    qubo_problem = QuadraticProgram()

    # Add binary variables for each edge
    for i in range(len(edge_list)):
        qubo_problem.binary_var(f"x_{i}")

    # Step 6: Create linear terms for the objective function
    linear_terms = {}
    for i, edge in enumerate(edge_list):
        u, v = edge
        distance = G[u][v]['weight']
        linear_terms[f"x_{i}"] = distance

    # Set the objective to minimize the total distance
    qubo_problem.minimize(linear=linear_terms)

    # Step 7: Add constraints to ensure that target edges are included
    for edge in target_edges:
        edge_sorted = tuple(sorted(edge))  # Normalize edge tuple
        if edge_sorted in edge_to_index:
            i = edge_to_index[edge_sorted]
            qubo_problem.linear_constraint(
                linear={f"x_{i}": 1},
                sense='==',
                rhs=1,
                name=f"target_edge_constraint_{i}"
            )
        else:
            # Handle cases where the target edge is not in the graph
            raise ValueError(f"Target edge {edge} not found in the graph.")

    # Step 8: Add penalties for optional edges to discourage their selection
    revisit_penalty = 10  # High penalty for optional edges
    for edge in optional_edges:
        edge_sorted = tuple(sorted(edge))  # Normalize edge tuple
        if edge_sorted in edge_to_index:
            i = edge_to_index[edge_sorted]
            qubo_problem.minimize(quadratic={(f"x_{i}", f"x_{i}"): revisit_penalty})
        else:
            # Handle cases where the optional edge is not in the graph
            pass  # Optional edges not in the graph can be ignored

    # Step 9: Solve the QUBO problem using QAOA
    sampler = Sampler()
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=200))
    optimizer = MinimumEigenOptimizer(qaoa)
    result = optimizer.solve(qubo_problem)

    # Step 10: Extract selected edges from the result
    selected_edges = []
    for i, edge in enumerate(edge_list):
        if result.x[i] == 1:
            selected_edges.append(edge)

    # Output the selected edges
    selected_edges_output = [(u, v) for u, v in selected_edges]

    # Step 11: Ensure the path is connected and find the traversal order starting from the first node
    selected_graph = nx.Graph()
    selected_graph.add_edges_from((u, v, G.get_edge_data(u, v)) for u, v in selected_edges)

    # Step 12: DFS traversal starting at the first node
    start_node = nodes[0]  # Start traversal from the first node
    if start_node in selected_graph.nodes:
        traversal_order = dfs_traversal_from_start(selected_graph, start_node)
        traversal_order_output = [(u, v) for u, v in traversal_order]
    else:
        traversal_order_output = []
        print(f"Node {start_node} is not in the selected subgraph. No traversal possible.")

    # Step 13: Create the output result
    output_result = {
        "selected_edges": selected_edges_output,
        "traversal_order": traversal_order_output
    }

    custom_circuit = create_custom_circuit(edge_list)

    # Step 14: Return the result as a JSON-compatible object
    return json.dumps(output_result)

# Helper function for DFS traversal
def dfs_traversal_from_start(graph, start_node):
    traversal_order = []
    visited_edges = set()

    def dfs(node):
        for neighbor in graph.neighbors(node):
            edge = (node, neighbor)
            edge_sorted = tuple(sorted(edge))
            if edge_sorted not in visited_edges:
                visited_edges.add(edge_sorted)
                traversal_order.append(edge)
                dfs(neighbor)

    dfs(start_node)
    return traversal_order




In [21]:
!pip install matplotlib
!pip install pylatexenc

To calculate the cost function for your QUBO problem, you'll need a function that evaluates the energy of the current solution (i.e., the binary variable values x_i that are either 0 or 1) based on the QUBO's objective function.

Key Steps for Cost Function Calculation:
Objective Function: The QUBO objective function is typically of the form:

𝐸
(
𝑥
)
=
∑
𝑖
𝑐
𝑖
𝑥
𝑖
+
∑
𝑖
,
𝑗
𝑄
𝑖
𝑗
𝑥
𝑖
𝑥
𝑗
E(x)=
i
∑
​
 c
i
​
 x
i
​
 +
i,j
∑
​
 Q
ij
​
 x
i
​
 x
j
​

Where:

𝑐
𝑖
c
i
​
  are the linear coefficients (for individual variables).
𝑄
𝑖
𝑗
Q
ij
​
  are the quadratic coefficients (for pairs of variables).
Cost Calculation: The cost function should sum up the contributions from the linear and quadratic terms based on the current solution (x values).

Penalties: If there are constraints (like penalties for selecting optional edges), you should include those in the cost calculation as well.

Example of Adding a Cost Calculation Function
Here’s how you can define a cost function based on your QUBO problem:

In [27]:
# from chatGPT

# Function to calculate the cost of a given solution
def calculate_cost(solution, qubo_problem, G, edge_list):
    cost = 0

    # Step 1: Linear terms (individual variables)
    for i, (u, v) in enumerate(edge_list):
        if solution[i] == 1:  # Check if edge (u, v) is selected
            cost += G[u][v]['weight']  # Add the weight of the edge to the cost

    # Step 2: Quadratic terms (interaction between variables)
    for i, (u1, v1) in enumerate(edge_list):
        for j, (u2, v2) in enumerate(edge_list):
            if i != j and solution[i] == 1 and solution[j] == 1:
                # Add a penalty for selecting both edges (if needed, e.g., revisiting penalty)
                revisit_penalty = 10  # Define your penalty
                cost += revisit_penalty  # Add penalty for both edges being selected

    return cost

In [33]:
# From ChatGPT
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
import networkx as nx
import json

# Define a custom quantum circuit
def create_custom_circuit(num_qubits):
    qc = QuantumCircuit(num_qubits)
    theta = Parameter('θ')  # Parameter for RX gate
    for i in range(num_qubits):
        qc.h(i)  # Apply Hadamard gate to each qubit
    for i in range(num_qubits):
        qc.rx(theta, i)  # Add RX gate to each qubit
        qc.rx(theta, i)  # Add RX gate to each qubit

    qc.draw(output='mpl')
    print(qc)
    return qc

# Main function with QAOA and custom circuit
def run_with_custom_circuit(input_data, solver_params=None, extra_arguments=None):
    nodes = input_data['nodes']
    adjacency_matrix = input_data['adjacency_matrix']
    target_edges = input_data['target_edges']
    optional_edges = input_data['optional_edges']

    G = nx.Graph()
    G.add_nodes_from(nodes)

    n = len(nodes)
    node_indices = {i: node for i, node in enumerate(nodes)}  # Map index to node label
    for i in range(n):
        for j in range(n):
            if adjacency_matrix[i][j] > 0:
                u = node_indices[i]
                v = node_indices[j]
                G.add_edge(u, v, weight=adjacency_matrix[i][j])

    edge_list = list(G.edges())
    edge_to_index = {tuple(sorted(edge)): i for i, edge in enumerate(edge_list)}

    qubo_problem = QuadraticProgram()
    for i in range(len(edge_list)):
        qubo_problem.binary_var(f"x_{i}")

    linear_terms = {f"x_{i}": G[u][v]['weight'] for i, (u, v) in enumerate(edge_list)}
    qubo_problem.minimize(linear=linear_terms)

    for edge in target_edges:
        edge_sorted = tuple(sorted(edge))
        if edge_sorted in edge_to_index:
            i = edge_to_index[edge_sorted]
            qubo_problem.linear_constraint(
                linear={f"x_{i}": 1},
                sense='==',
                rhs=1,
                name=f"target_edge_constraint_{i}"
            )
        else:
            raise ValueError(f"Target edge {edge} not found in the graph.")

    revisit_penalty = 10
    for edge in optional_edges:
        edge_sorted = tuple(sorted(edge))
        if edge_sorted in edge_to_index:
            i = edge_to_index[edge_sorted]
            qubo_problem.minimize(quadratic={(f"x_{i}", f"x_{i}"): revisit_penalty})

    # Custom quantum circuit and QAOA
    num_qubits = len(edge_list)
    custom_circuit = create_custom_circuit(num_qubits)

    sampler = Sampler()
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=200), mixer=custom_circuit)
    optimizer = MinimumEigenOptimizer(qaoa)
    result = optimizer.solve(qubo_problem)
    qaoa.draw(output='mpl')
    print(qaoa)

    selected_edges = [edge_list[i] for i in range(len(edge_list)) if result.x[i] == 1]

   # Calculate the cost of the result
    cost = calculate_cost(result.x, qubo_problem, G, edge_list)
    print(f"Cost of the solution: {cost}")

    # Traversal and Output
    selected_edges_output = [(u, v) for u, v in selected_edges]
    selected_graph = nx.Graph()
    selected_graph.add_edges_from((u, v, G.get_edge_data(u, v)) for u, v in selected_edges)

    start_node = nodes[0]
    traversal_order_output = dfs_traversal_from_start(selected_graph, start_node) if start_node in selected_graph.nodes else []

    output_result = {
        "selected_edges": selected_edges_output,
        "traversal_order": traversal_order_output
    }

    return json.dumps(output_result)

# Helper function for DFS traversal
def dfs_traversal_from_start(graph, start_node):
    traversal_order = []
    visited_edges = set()

    def dfs(node):
        for neighbor in graph.neighbors(node):
            edge = (node, neighbor)
            edge_sorted = tuple(sorted(edge))
            if edge_sorted not in visited_edges:
                visited_edges.add(edge_sorted)
                traversal_order.append(edge)
                dfs(neighbor)

    dfs(start_node)
    return traversal_order


In [17]:
import json
import csv
#from google.colab import files
#uploaded = files.upload()

Saving presenTest.json to presenTest (1).json


In [34]:
# read the json file and save it as a pandas.Dataframe
input_data = pd.read_json('presenTest.json')
input_data

,data
nodes,"[1, 2, 3, 4, 5, 6, 7, 8]"
target_edges,"[[1, 3], [1, 5], [3, 4], [4, 5]]"
optional_edges,"[[1, 2], [2, 6], [5, 6], [5, 7], [7, 3]]"
adjacency_matrix,"[[0, 1, 2, 0, 2, 0, 0, 0], [1, 0, 0, 0, 0, 1, ..."


In [35]:
output_data2 = run_with_custom_circuit(input_data['data'])
output_data2

     ┌───┐┌───────┐┌───────┐
q_0: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     ├───┤├───────┤├───────┤
q_1: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     ├───┤├───────┤├───────┤
q_2: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     ├───┤├───────┤├───────┤
q_3: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     ├───┤├───────┤├───────┤
q_4: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     ├───┤├───────┤├───────┤
q_5: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     ├───┤├───────┤├───────┤
q_6: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     ├───┤├───────┤├───────┤
q_7: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     ├───┤├───────┤├───────┤
q_8: ┤ H ├┤ Rx(θ) ├┤ Rx(θ) ├
     └───┘└───────┘└───────┘


<ipython-input-33-7a598edec04a>:79: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


AttributeError: 'QAOA' object has no attribute 'draw'